## Updating and importing packages

Skit-learn and Implicit pakages have to be updated to work properly. If not they won't work. 

In [2]:
!pip install scikit_learn --upgrade
!pip install implicit

Requirement already up-to-date: scikit_learn in /Users/leilasapple/anaconda/lib/python3.6/site-packages (0.20.0)


In [3]:
# Importing required packeges
import pandas as pd
import numpy as np
import os,sys
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler
import random
from sklearn import metrics
import implicit 

/Users/leilasapple/anaconda/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/Users/leilasapple/.matplotlib/matplotlibrc", line #621
  (fname, cnt))
/Users/leilasapple/anaconda/lib/python3.6/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


## Define functions

In [4]:
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered 

In [5]:
def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 10):
    '''
    This function will return the top recommended items to our users 
    
    parameters:
    
    customer_id - Input the customer's id number that you want to get recommendations for
    
    mf_train - The training matrix you used for matrix factorization fitting
    
    user_vecs - the user vectors from your fitted matrix factorization
    
    item_vecs - the item vectors from your fitted matrix factorization
    
    customer_list - an array of the customer's ID numbers that make up the rows of your ratings matrix 
                    (in order of matrix)
    
    item_list - an array of the products that make up the columns of your ratings matrix
                    (in order of matrix)
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    num_items - The number of items you want to recommend in order of best recommendations. Default is 10. 
    
    returns:
    
    - The top n recommendations chosen based on the user/item vectors for items never interacted with/purchased
    '''
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    pref_vec = mf_train[cust_ind,:].toarray() # Get the ratings from the training set ratings matrix
    pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that items not purchased yet become equal to 1
    pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T) # Get dot product of user vector and all item vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled 
    # Items already purchased have their recommendation multiplied by zero
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] # Sort the indices of the items into order 
    # of best recommendations
    rec_list = [] # start empty list to store items
    for index in product_idx:
        code = item_list[index]
        rec_list.append([code, item_lookup.product_name.loc[item_lookup.product_id == code].iloc[0]]) 
        # Append our descriptions to the list
    codes = [item[0] for item in rec_list]
    descriptions = [item[1] for item in rec_list]
    final_frame = pd.DataFrame({'product_id': codes, 'product_name': descriptions}) # Create a dataframe 
    return final_frame[['product_id', 'product_name']] # Switch order of columns around

In [6]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    
    return metrics.auc(fpr, tpr)   

In [7]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

## Input  directoies and files

In [8]:
#The input directory in the google dirve
input_data_path="../Fromgoogle/Input_files"
os.listdir(input_data_path+'/produced')

['.ipynb_checkpoints',
 'data_InstaCart.csv',
 'data_InstaCart_rating.csv',
 'data_InstaCart_sparse.npz',
 'data_InstaCart_sparse_2.npz']

In [9]:
purchases_sparse=sparse.load_npz(os.path.join(input_data_path+'/produced/','data_InstaCart_sparse_2.npz'))
#rating=pd.read_csv(os.path.join(input_data_path+'/produced/','data_InstaCart_rating.csv'))

## Main program

### Train and test datasets

In [10]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)

### Model

In [11]:
alpha = 15
user_vecs, item_vecs = implicit.alternating_least_squares((product_train*alpha).astype('double'), 
                                                          factors=20, 
                                                          regularization = 0.1, 
                                                         iterations = 50)

This method is deprecated. Please use the AlternatingLeastSquares class instead
100%|██████████| 50.0/50 [47:22<00:00, 62.36s/it]


In [12]:
print(type(user_vecs),user_vecs.shape,item_vecs.shape)

<class 'numpy.ndarray'> (206209, 20) (49688, 20)


In [13]:
np.savetxt(os.path.join(input_data_path,'user_vecs.csv'),user_vecs, delimiter=",")
np.savetxt(os.path.join(input_data_path,'item_vecs.csv'),item_vecs, delimiter=",")

In [14]:
data=pd.read_csv(os.path.join(input_data_path+'/produced','data_InstaCart.csv'))[['user_id','product_id']]

In [15]:
#data=data.join(pd.DataFrame({'user_quantity':data.groupby('user_id').size()})
#                                 .reset_index().set_index('user_id'),on=('user_id'))


In [16]:
item_lookup=pd.read_csv(os.path.join('../Input_files/','products.csv'))[['product_id','product_name']]
item_lookup.head()

,product_id,product_name
0,1,Chocolate Sandwich Cookies
1,2,All-Seasons Salt
2,3,Robust Golden Unsweetened Oolong Tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...
4,5,Green Chile Anytime Sauce


In [17]:
customers_arr = np.array(data.user_id.unique()) # Array of customer IDs from the ratings matrix

In [18]:
products_arr = np.array(data.product_id.unique()) # Array of product IDs from the ratings matrix

In [19]:
customers_arr.max()

206209

In [20]:
products_arr[:10]

array([   196.,  12427.,  14084.,  26088.,  26405.,  10258.,  13032.,
        13176.,  25133.,  30450.])

In [21]:
#rec_items(1, purchases_sparse, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,num_items = 10)
id=5
print('\033[1mThe products that this customer has orderd so far:\033[0m\n\n',
    item_lookup.product_name[item_lookup.product_id.isin(data.loc[data.user_id==id,'product_id'].unique().astype(int))],
      '\033[1m\n\nThe products that similar customers liked:\033[0m\n',
      rec_items(id, purchases_sparse, user_vecs, item_vecs, customers_arr, products_arr, item_lookup
                ,num_items = 20))

The products that this customer has orderd so far:

 3375     Organic Whole Kernel Sweet Corn No Salt Added
5998                         Semi-Soft Cheese, Ripened
6807               Organic Spinach & Cheese Tortellini
8517                                 Organic Red Onion
11776                                  Red Raspberries
13869             Lightly Salted Baked Snap Pea Crisps
13987                                        Snow Peas
15348                         Organic Raw Agave Nectar
16167                  Large Organic Omega3 Brown Eggs
16184                             Sharp Cheddar Cheese
18760                       Balsamic Vinegar Of Modena
19056             Organic Large Extra Fancy Fuji Apple
20113                                 Jalapeno Peppers
20753                        Mediterranean Mint Gelato
20842               100% Pure Eucalyptus Essential Oil
21412                                     Organic Soba
21615                             Organic Baby Arugula
22474       

### Evaluation, AUC

In [22]:
roc=calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)

In [23]:
print('The mean AUC:',roc[0])

The mean AUC: 0.961
